In [ ]:
# 1º Passo – Importar as bibliotecas que serão utilizadas

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pandas as pd
import seaborn as sn
%matplotlib inline

In [ ]:
# 2º Passo – Carregar os arrays construídos na etapa referente ao pré-processamento de dados e normalizar os dados do input

X_train = np.load("/content/drive/My Drive/Python/COVID/Arrays/Modelo2/X_Train.npy")
X_train = X_train/255

Y_train = np.load("/content/drive/My Drive/Python/COVID/Arrays/Modelo2/Y_Train.npy")

In [ ]:
# 3º Passo – Dividir os dados em dados de treinamento e dados de teste

X_train,X_test,Y_train,Y_test = train_test_split(X_train,Y_train, test_size = 0.2, random_state = 40)

print(f"X_train shape: {X_train.shape} Y_train shape {Y_train.shape}")
print(f"X_test shape: {X_test.shape} Y_test shape {Y_test.shape}")

In [ ]:
# 4º Passo – Determinando a arquitetura do modelo (Xception) que será treinado

bModel = Xception(weights="imagenet", include_top=False,
        input_tensor=Input(shape=(237, 237, 3)))
tModel = bModel.output
tModel = AveragePooling2D(pool_size=(2, 2))(tModel)
tModel = Flatten(name="flatten")(tModel)
tModel = Dense(20, activation="relu")(tModel)
tModel = Dropout(0.2)(tModel)
tModel = Dense(3, activation="softmax")(tModel)

model = Model(inputs=bModel.input, outputs=tModel)

In [ ]:
# 5º Passo – Determinar os hyperparameters e compilar o modelo (Xception)

INIT_LR = 1e-3
EPOCHS = 80
BS = 15

for layer in bModel.layers:
    layer.trainable = False

opt = Adam(lr=INIT_LR)
model.compile(loss="categorical_crossentropy", optimizer=opt,
    metrics=["accuracy"])

In [ ]:
# 6º Passo – Treinar o modelo (Xception)

from tensorflow.keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001, cooldown=5)

x = model.fit(X_train, Y_train, batch_size=BS,validation_split=0.1, epochs=EPOCHS,callbacks=[reduce_lr])

model.save("/content/drive/My Drive/Python/COVID/model/modeloc_2.hdf5")

In [ ]:
# 7º Passo - Observar a acurácia do modelo (Xception) e a função de perda

plt.plot(x.history['accuracy'])
plt.plot(x.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.plot(x.history['loss'])
plt.plot(x.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

model.evaluate(X_test,Y_test)

In [ ]:
# 8º Passo – Determinando a arquitetura do modelo (ResNet50V2) que será treinado

bModel = ResNet50V2(weights="imagenet", include_top=False,
        input_tensor=Input(shape=(237, 237, 3)))
tModel = bModel.output
tModel = AveragePooling2D(pool_size=(2, 2))(tModel)
tModel = Flatten(name="flatten")(tModel)
tModel = Dense(20, activation="relu")(tModel)
tModel = Dropout(0.2)(tModel)
tModel = Dense(3, activation="softmax")(tModel)

model = Model(inputs=bModel.input, outputs=tModel)

In [ ]:
# 9º Passo – Determinar os hyperparameters e compilar o modelo (ResNet50V2)

INIT_LR = 1e-3
EPOCHS = 80
BS = 15

for layer in bModel.layers:
	layer.trainable = False

opt = Adam(lr=INIT_LR)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

In [ ]:
# 10º Passo – Treinar o modelo (ResNet50V2)

from tensorflow.keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001, cooldown=5)

x = model.fit(X_train, Y_train, batch_size=BS,validation_split=0.1, epochs=EPOCHS,callbacks=[reduce_lr])

model.save("/content/drive/My Drive/Python/COVID/model/modeloc_2.hdf5")

In [ ]:
# 11º Passo - Observar a acurácia do modelo (ResNet50V2) e a função de perda

plt.plot(x.history['accuracy'])
plt.plot(x.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.plot(x.history['loss'])
plt.plot(x.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

model.evaluate(X_test,Y_test)

In [ ]:
# 12º Passo – Determinando a arquitetura (VGG16) do modelo que será treinado

bModel = VGG16(weights="imagenet", include_top=False,classes=3,
        input_tensor=Input(shape=(237, 237, 3)))
tModel = bModel.output
tModel = AveragePooling2D(pool_size=(2, 2))(tModel)
tModel = Flatten(name="flatten")(tModel)
tModel = Dense(20, activation="relu")(tModel)
tModel = Dropout(0.2)(tModel)
tModel = Dense(3, activation="softmax")(tModel)

model = Model(inputs=bModel.input, outputs=tModel)

In [ ]:
# 13º Passo – Determinar os hyperparameters e compilar o modelo (VGG16)

INIT_LR = 1e-3
EPOCHS = 80
BS = 15

for layer in bModel.layers:
	layer.trainable = False

opt = Adam(lr=INIT_LR)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

In [ ]:
# 14º Passo – Treinar o modelo (VGG16)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001, cooldown=5)

x = model.fit(X_train, Y_train, batch_size=BS,validation_split=0.1, epochs=EPOCHS,callbacks=[reduce_lr])

model.save("/content/drive/My Drive/Python/COVID/model/modeloc_2.hdf5")

In [ ]:
# 15º Passo – Observar a acurácia do modelo (VGG16) e a função de perda

plt.plot(x.history['accuracy'])
plt.plot(x.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.plot(x.history['loss'])
plt.plot(x.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

model.evaluate(X_test,Y_test)

In [ ]:
# 16º Passo – Observar quais imagens o modelo (VGG16) acertou

plt.figure(figsize=(20,20))
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=2.0, hspace=2.0)
i = 0
for i,image in enumerate(X_test):
    plt.subplot(9,9,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(image, cmap=plt.cm.binary)
    img = np.expand_dims(X_test[i],axis = 0)
    x_pred = model.predict(img)[0]
    pred_covid = x_pred[0]
   
    pred_normal = x_pred[1]

    pred_infeccoes = x_pred[2]
    
    
    if pred_covid > pred_normal and pred_covid > pred_infeccoes:
      label = "COVID"
    elif pred_normal > pred_covid and pred_normal > pred_infeccoes:
      label = "NORMAL"
    elif pred_infeccoes > pred_covid and pred_infeccoes > pred_normal:
      label = "INFECÇÕES"
     
    
    if Y_test[i][0] == 1:
      label_test = "COVID"
    elif Y_test[i][1] == 1:
      label_test = "NORMAL"
    elif Y_test[i][2] == 1:
      label_test = "INFECÇÕES"
    plt.xlabel(f"Label Predict = {label} \n Label Correct = {label_test}")
    i += 1
plt.savefig('/content/drive/My Drive/Python/COVID/model/modelo_2.pdf')
